# Projeto Final de IA: Classificador de Dificuldade de Questões do ENEM
---
**Aluna:** Ana Laura Tavares Costa

**Disciplina:** INF 420 - Inteligência Artificial I

**Professor:** Julio Cesar Soares dos Reis

**Objetivo do Projeto:** Comparar três modelos de Machine Learning para classificar questões de Matemática do ENEM por nível de dificuldade (Fácil, Médio, Difícil), com base nos microdados públicos do INEP.


## **Carregamento dos Dados**

---

**Atenção:** Para executar esta etapa, é necessário que os arquivos de dados do ENEM de 2022 e 2023 já tenham sido baixados e estejam no seu Google Drive.

Este notebook espera que os seguintes arquivos estejam localizados na **pasta raiz do seu Google Drive ('Meu Drive')**:
1.  `MICRODADOS_ENEM_2022.csv` (contém as respostas dos participantes)
2.  `ITENS_PROVA_2022.csv` (contém as informações sobre cada questão da prova)
3.  `MICRODADOS_ENEM_2023.csv` (contém as respostas dos participantes)
4.  `ITENS_PROVA_2023.csv` (contém as informações sobre cada questão da prova)

Caso ainda não tenha os dados, eles podem ser baixados do portal oficial do INEP:
* **Link para Download:** [Microdados do ENEM - INEP](https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/microdados/enem)

O código a seguir irá conectar com seu Google Drive e carregar os dois arquivos em DataFrames separados.

In [ ]:
import pandas as pd
import numpy as np
import gc
import os

# modelos e ferramentas de avaliação
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# conexão com google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


# função que processa os dados de um ano do ENEM para extrair as questões de matemática da aplicação regular com seus parâmetros da TRI
# retorna um dataframe limpo para ser utilizado ao longo do projeto
def processar_dados_enem_anual(ano, pasta_drive):
    print(f"--- Processando dados do ano: {ano} ---")

    path_itens = os.path.join(pasta_drive, f'ITENS_PROVA_{ano}.csv')
    path_microdados = os.path.join(pasta_drive, f'MICRODADOS_ENEM_{ano}.csv')

    try:
        # carrega os dados do ano específico
        df_itens = pd.read_csv(path_itens, sep=';', encoding='latin-1')
        df_microdados = pd.read_csv(path_microdados, sep=';', encoding='latin-1', usecols=['CO_PROVA_MT', 'TP_PRESENCA_MT', 'TX_RESPOSTAS_MT', 'TX_GABARITO_MT'])

        # identifica os códigos da prova regular
        df_microdados_presentes = df_microdados[df_microdados['TP_PRESENCA_MT'] == 1]
        contagem_provas = df_microdados_presentes['CO_PROVA_MT'].value_counts()
        codigos_provas_regulares = contagem_provas.head(4).index.tolist()
        print(codigos_provas_regulares)

        # filtra para obter as questões de interesse
        df_analise_ano = df_itens[
            (df_itens['SG_AREA'] == 'MT') &
            (df_itens['IN_ITEM_ABAN'] == 0) &
            (df_itens['TX_COR'] == 'AZUL') &
            (df_itens['CO_PROVA'].isin(codigos_provas_regulares))
        ].copy()

        # remove duplicatas para ter uma linha por questão única
        df_analise_ano = df_analise_ano.drop_duplicates(subset=['CO_ITEM'])

        print(f"Ano {ano} processado com sucesso. Encontradas {len(df_analise_ano)} questões válidas.")
        return df_analise_ano, df_microdados_presentes

    except FileNotFoundError:
        print(f"Arquivos para o ano {ano} não encontrados.")
        return None

### Identificação das provas regulares


In [ ]:
pasta_drive = '/content/drive/MyDrive/'
anos_para_processar = [2022, 2023]
lista_dataframes_anuais = []
lista_df_microdados = []

for ano in anos_para_processar:
    df_ano, df_md = processar_dados_enem_anual(ano, pasta_drive)
    if df_ano is not None and df_md is not None:
        lista_dataframes_anuais.append(df_ano)
        lista_df_microdados.append(df_md)

# junta os dataframes dos anos em um único
if lista_dataframes_anuais:
    df_itens_total = pd.concat(lista_dataframes_anuais, ignore_index=True)
    df_md_total = pd.concat(lista_df_microdados, ignore_index=True)
    print("\n--- Processamento Concluído ---")
    print(f"Dataset final para modelagem criado com {len(df_itens_total)} questões no total.")
    display(df_itens_total.head())
    display(df_md_total.head())
else:
    print("\nNenhum dado foi processado. Verifique os caminhos dos arquivos.")

In [ ]:
print(len(df_itens_total))
print(len(df_md_total))

### Engenharia de Atributos

In [ ]:
if 'df_itens_total' in locals() and not df_itens_total.empty:
    # criação do alvo (y)
    rotulos = ['Fácil', 'Média', 'Difícil']
    df_itens_total['dificuldade_oficial'] = pd.qcut(df_itens_total['NU_PARAM_B'], q=3, labels=rotulos)
    y = df_itens_total['dificuldade_oficial']

    # seleção e preparação dos atributos (X)
    features_iniciais = df_itens_total[['NU_PARAM_A', 'NU_PARAM_C', 'CO_HABILIDADE']]
    X = pd.get_dummies(features_iniciais, columns=['CO_HABILIDADE'], prefix='HAB')

    print("Dataset final (X e y) preparado para a avaliação dos modelos.")
    display(X)
    display(y)
    display(df_itens_total)
else:
    print("DataFrame 'df_itens_total' não foi criado.")

### Treinamento e Avaliação do Modelo Baseline

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# verifica se X e y existem antes de prosseguir
if 'X' in locals() and 'y' in locals():
    # dicionário com os modelos a serem testados
    modelos = {
        'Regressão Logística': LogisticRegression(max_iter=1000, random_state=42),
        'Random Forest': RandomForestClassifier(random_state=42),
        'SVM (Support Vector Machine)': SVC(random_state=42)
    }

    metricas_de_avaliacao = ['accuracy', 'f1_weighted']

    print("--- Avaliando Modelos com Validação Cruzada (k=5) ---")

    # loop para criar um pipeline e avaliar cada modelo
    for nome, modelo in modelos.items():
        # garante que a padronização é feita corretamente em cada fold
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('classifier', modelo)
        ])

        scores = cross_validate(pipeline, X, y, cv=5, scoring=metricas_de_avaliacao)

        acc_media = scores['test_accuracy'].mean()
        acc_std = scores['test_accuracy'].std()
        f1_media = scores['test_f1_weighted'].mean()
        f1_std = scores['test_f1_weighted'].std()

        print(f"\nModelo: {nome}")
        print(f"Acurácia Média: {acc_media:.4f} (Desvio Padrão: {acc_std:.4f})")
        print(f"F1-Score Ponderado Médio: {f1_media:.4f} (Desvio Padrão: {f1_std:.4f})")
        print("-" * 50)

else:
    print("Variáveis X e y não foram definidas. Execute a célula anterior.")

## **Adicionando novos features para melhorar a performance dos modelos**

#### Preparação


Serão selecionados os alunos que efetivamente fizeram as provas que filtramos na etapa de filtragem de provas válidas

In [ ]:
# verifica se os dataframes de partida existem
if 'df_itens_total' in locals() and 'df_md_total' in locals():

    # extrair a lista de códigos de prova de interesse
    # .unique().tolist() garante que pegamos cada código de prova apenas uma vez
    print(df_itens_total['CO_PROVA'])
    codigos_provas_relevantes = df_itens_total['CO_PROVA'].unique().tolist()

    print(f"Identificados {len(codigos_provas_relevantes)} códigos de prova correspondentes às questões de interesse.")
    print(codigos_provas_relevantes)

    # filtrar os alunos que fizeram essas provas
    print(f"\nFiltrando o DataFrame de {len(df_md_total)} alunos...")

    df_alunos_relevantes = df_md_total[
        df_md_total['CO_PROVA_MT'].isin(codigos_provas_relevantes)
    ].copy()

    print(f"Encontrados {len(df_alunos_relevantes)} alunos que fizeram as provas de interesse.")

    # cria a amostra aleatória de 100.000 alunos
    tamanho_amostra = 100000

    # garante que temos alunos suficientes para a amostragem
    if len(df_alunos_relevantes) >= tamanho_amostra:
        print(f"\nCriando uma amostra aleatória de {tamanho_amostra} alunos...")
        df_amostra_alunos = df_alunos_relevantes.sample(n=tamanho_amostra, random_state=42)

        print("Amostra final criada com sucesso!")
        display(df_amostra_alunos.head())
    else:
        print(f"\nAVISO: O número de alunos relevantes ({len(df_alunos_relevantes)}) é menor que o tamanho da amostra desejado. Usando todos os alunos encontrados.")
        df_amostra_alunos = df_alunos_relevantes


    del df_md_total
    del df_alunos_relevantes
    import gc
    gc.collect()
    print("\nMemória liberada.")

else:
    print("DataFrames 'df_itens_total' ou 'df_md_total' não foram encontrados. Execute as células de carregamento.")

### Processamento

#### Preparação da Estrutura de Contagem

In [ ]:
# verifica se o dataframe de itens existe
if 'df_itens_total' in locals():
    # cria o dicionário que armazenará as contagens
    contagem_features = {}

    # pega a lista de todas as questões únicas que vamos analisar
    lista_itens_unicos = df_itens_total['CO_ITEM'].unique()

    # inicia o "placar" para cada questão com os contadores zerados
    for item_code in lista_itens_unicos:
        contagem_features[item_code] = {'acertos': 0, 'brancos': 0, 'total_participacoes': 0}

    print(f"Estrutura de contagem criada para {len(contagem_features)} questões.")
    print("Pronto para iniciar o processamento.")
else:
    print("DataFrame 'df_itens_total' não foi criado. Execute as células anteriores.")

#### Contagem de Acertos Por Questão e Deixadas em Branco

In [ ]:
if 'contagem_features' in locals():
    print("Iniciando o processamento da amostra de alunos (com gabarito corrigido)...")
    total_alunos = len(df_amostra_alunos)

    for i, aluno in enumerate(df_amostra_alunos.itertuples()):
        if (i + 1) % 10000 == 0:
            print(f"Processando aluno {i + 1} de {total_alunos}...")

        cod_prova_aluno = aluno.CO_PROVA_MT
        respostas_aluno = aluno.TX_RESPOSTAS_MT

        itens_da_prova = df_itens_total[df_itens_total['CO_PROVA'] == cod_prova_aluno]
        itens_da_prova = itens_da_prova.sort_values(by='CO_POSICAO')

        for indice_relativo, item in enumerate(itens_da_prova.itertuples()):
            cod_item = item.CO_ITEM

            if cod_item in contagem_features:
                contagem_features[cod_item]['total_participacoes'] += 1

                posicao_na_string = indice_relativo
                resposta = respostas_aluno[posicao_na_string]

                gabarito = item.TX_GABARITO

                if resposta == '.':
                    contagem_features[cod_item]['brancos'] += 1
                else:
                    if resposta == gabarito:
                        contagem_features[cod_item]['acertos'] += 1

    print(f"\nProcessamento concluído!")

    df_features_avancadas = pd.DataFrame.from_dict(contagem_features, orient='index')
    respostas_preenchidas = df_features_avancadas['total_participacoes'] - df_features_avancadas['brancos']
    df_features_avancadas['taxa_acerto'] = (df_features_avancadas['acertos'] / respostas_preenchidas).fillna(0)
    df_features_avancadas['taxa_em_branco'] = (df_features_avancadas['brancos'] / df_features_avancadas['total_participacoes']).fillna(0)
    df_features_avancadas = df_features_avancadas[['taxa_acerto', 'taxa_em_branco']]
    df_features_avancadas.index.name = 'CO_ITEM'

    print("\nDataFrame com as novas features:")
    display(df_features_avancadas.head())
else:
    print("Estruturas de dados necessárias não foram encontradas.")

#### Pós-processamento e Criação das Features Finais

In [ ]:
if 'contagem_features' in locals() and any(contagem_features):
    df_features_avancadas = pd.DataFrame.from_dict(contagem_features, orient='index')

    # calcula as respostas que não foram deixadas em branco
    respostas_preenchidas = df_features_avancadas['total_participacoes'] - df_features_avancadas['brancos']

    # calcula a taxa de acerto, tratando a divisão por zero (caso uma questão só tenha tido respostas em branco)
    df_features_avancadas['taxa_acerto'] = (df_features_avancadas['acertos'] / respostas_preenchidas).fillna(0)

    # calcula a taxa de respostas em branco
    df_features_avancadas['taxa_em_branco'] = (df_features_avancadas['brancos'] / df_features_avancadas['total_participacoes']).fillna(0)

    # seleciona apenas as colunas finais que nos interessam e define o índice
    df_features_avancadas = df_features_avancadas[['taxa_acerto', 'taxa_em_branco']]
    df_features_avancadas.index.name = 'CO_ITEM'

    print("DataFrame com as novas features ('taxa_acerto' e 'taxa_em_branco') foi calculado com sucesso:")
    display(df_features_avancadas.head())
else:
    print("Dicionário 'contagem_features' está vazio ou não existe. O loop pode não ter sido executado corretamente.")

#### Junção dos features e preparação para ML

In [ ]:
if 'df_itens_total' in locals() and 'df_features_avancadas' in locals():
    # define o índice para a junção correta
    df_base = df_itens_total.set_index('CO_ITEM')
    df_novas_features = df_features_avancadas.copy()
    df_novas_features.index = pd.to_numeric(df_novas_features.index)

    # cria a coluna alvo 'dificuldade_oficial'
    rotulos = ['Fácil', 'Média', 'Difícil']
    df_base['dificuldade_oficial'] = pd.qcut(df_base['NU_PARAM_B'], q=3, labels=rotulos)

    features_base = df_base[['NU_PARAM_A', 'NU_PARAM_C', 'CO_HABILIDADE']]
    features_base_encoded = pd.get_dummies(features_base, columns=['CO_HABILIDADE'], prefix='HAB')

    # junta as features de base com as avançadas
    X_avancado = pd.merge(
        left=features_base_encoded,
        right=df_novas_features,
        left_index=True,
        right_index=True,
        how='inner'
    )

    # junta as features finais com o alvo
    df_modelagem_final = pd.concat([X_avancado, df_base['dificuldade_oficial']], axis=1).dropna()

    print("DataFrame final pronto para modelagem!")
    display(df_modelagem_final.head())
else:
    print("Um dos DataFrames de partida não foi encontrado.")

### Avaliação Final

In [ ]:
if 'df_modelagem_final' in locals() and not df_modelagem_final.empty:

    # separa as features (X) e o alvo (y)
    X_final = df_modelagem_final.drop('dificuldade_oficial', axis=1)
    y_final = df_modelagem_final['dificuldade_oficial']

    # dicionário com os modelos
    modelos = {
        'Regressão Logística': LogisticRegression(max_iter=1000, random_state=42),
        'Random Forest': RandomForestClassifier(random_state=42),
        'SVM (Support Vector Machine)': SVC(random_state=42)
    }

    # define as métricas
    metricas_de_avaliacao = ['accuracy', 'f1_weighted']

    print("--- Avaliando Modelos com o Conjunto de Features Corrigido ---")

    # loop para avaliar cada modelo
    for nome, modelo in modelos.items():
        pipeline = Pipeline([ ('scaler', StandardScaler()), ('classifier', modelo) ])
        scores = cross_validate(pipeline, X_final, y_final, cv=5, scoring=metricas_de_avaliacao)

        acc_media = scores['test_accuracy'].mean()
        f1_media = scores['test_f1_weighted'].mean()

        print(f"\nModelo: {nome}")
        print(f"Acurácia Média: {acc_media:.4f}")
        print(f"F1-Score Ponderado Médio: {f1_media:.4f}")
        print("-" * 50)
else:
    print("DataFrame 'df_modelagem_final' não foi criado.")

### Ajuste Fino e Comparação Definitiva dos Modelos

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import pandas as pd

# verifica se o dataframe de modelagem final existe
if 'df_modelagem_final' in locals() and not df_modelagem_final.empty:

    # separa as features (X) e o alvo (y)
    X_final = df_modelagem_final.drop('dificuldade_oficial', axis=1)
    y_final = df_modelagem_final['dificuldade_oficial']

    # padroniza os dados, pois é necessário para LR e SVM
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_final)

    # grade para Regressão Logística
    param_grid_lr = {
        'C': [0.1, 1, 10, 100]
    }

    # grade para SVM
    param_grid_svm = {
        'C': [0.1, 1, 10, 100],
        'gamma': [1, 0.1, 0.01]
    }

    # grade para Random Forest
    param_grid_rf = {
        'n_estimators': [50, 100, 200],
        'max_depth': [5, 10, 20]
    }

    # lista contendo os modelos e seus parâmetros para iterar
    modelos_para_tuning = [
        ('Regressão Logística', LogisticRegression(max_iter=2000, random_state=42), param_grid_lr),
        ('SVM (Support Vector Machine)', SVC(random_state=42), param_grid_svm),
        ('Random Forest', RandomForestClassifier(random_state=42), param_grid_rf)
    ]

    # lista para guardar os melhores resultados de cada modelo
    melhores_resultados = []

    print("--- Iniciando Ajuste de Hiperparâmetros para Todos os Modelos ---")

    # itera sobre cada modelo e executa o GridSearchCV ---
    for nome, modelo, params in modelos_para_tuning:
        print(f"\n--- Ajustando: {nome} ---")

        # n_jobs=-1 usa todos os processadores para acelerar a busca
        grid_search = GridSearchCV(estimator=modelo, param_grid=params, cv=5, scoring='accuracy', n_jobs=-1)
        grid_search.fit(X_scaled, y_final) # usa os dados padronizados

        print(f"Melhores parâmetros encontrados: {grid_search.best_params_}")
        print(f"Melhor acurácia (com validação cruzada): {grid_search.best_score_:.4f}")

        melhores_resultados.append({
            'Modelo': nome,
            'Melhor Acurácia': grid_search.best_score_,
            'Melhores Parâmetros': grid_search.best_params_
        })

    # exibe a tabela para comparação
    print("\n\n--- Tabela Comparativa Final (Após Ajuste de Hiperparâmetros) ---")
    df_comparacao_final = pd.DataFrame(melhores_resultados)
    display(df_comparacao_final)

else:
    print("DataFrame 'df_modelagem_final' não foi criado.")

### Geração dos gráficos

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

dados_grafico = {
    'Modelo': ['Regressão Logística', 'Random Forest', 'SVM'] * 3,
    'Fase': ['1. Baseline'] * 3 + ['2. Features Avançadas'] * 3 + ['3. Otimizado'] * 3,
    'Acurácia': [
        0.3477, 0.4176, 0.3719,
        0.3941, 0.5000, 0.3601,
        0.4059, 0.5118, 0.3941
    ]
}
df_grafico = pd.DataFrame(dados_grafico)
plt.figure(figsize=(8, 5.2))
palette = "viridis"
ax = sns.barplot(data=df_grafico, x='Modelo', y='Acurácia', hue='Fase', palette=palette)

plt.ylabel('Acurácia Média (Validação Cruzada, k=5)', fontsize=14)
plt.xlabel('')
plt.ylim(0, 0.6)
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.legend(title='Fase do Projeto', fontsize=12, title_fontsize=13)

ax.tick_params(axis='x', labelsize=12)
ax.tick_params(axis='y', labelsize=12)

for p in ax.patches:
    if p.get_height() > 0:
        ax.annotate(format(p.get_height(), '.2%'),
                    (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='center',
                    xytext=(0, 8),
                    textcoords='offset points',
                    fontsize=11)

plt.tight_layout()

plt.savefig('grafico_etapas.pdf', bbox_inches='tight')
# plt.savefig('figuras/grafico_comparativo_300dpi.png', dpi=300, bbox_inches='tight')

plt.show()


### Importância de cada feature no Random Forest

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

if 'df_modelagem_final' in locals():
    print("Analisando a importância de cada feature para o melhor modelo (Random Forest)...")

    X_final = df_modelagem_final.drop('dificuldade_oficial', axis=1)
    y_final = df_modelagem_final['dificuldade_oficial']

    modelo_final_rf = Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', RandomForestClassifier(
            n_estimators=50,
            max_depth=20,
            random_state=42
        ))
    ])

    modelo_final_rf.fit(X_final, y_final)

    importancias = modelo_final_rf.named_steps['classifier'].feature_importances_
    nomes_features = X_final.columns

    df_importancia = pd.DataFrame({'feature': nomes_features, 'importance': importancias})
    df_importancia = df_importancia.sort_values(by='importance', ascending=False)

    plt.figure(figsize=(12, 10))
    sns.barplot(x='importance', y='feature', data=df_importancia.head(15), palette='viridis')

    plt.title('Top 15 Features Mais Importantes para o Modelo Random Forest', fontsize=16)
    plt.xlabel('Importância Relativa', fontsize=12)
    plt.ylabel('Feature', fontsize=12)
    plt.grid(axis='x', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

else:
    print("DataFrame 'df_modelagem_final' não foi encontrado. Execute as células de preparação e unificação primeiro.")